ORF Finding method:

The ORF finder is primarily composed of three functions:
-  codons(seq,start_c)
-  ORF(lst1,lst2,min_len)
-  ORFfinder(dta,nucleotides, sel_gen, rev,min_len):

codons(seq, start_c) will find all positions of start and stop codons, and place them into three lists, one for each frame. Once found, the ORF() function will accept these lists and validate/pair the start and stop pairs based on a set minimum length as well as where the previous ORF was found. For each valid start codon, multiple valid stop codons are paired in order to get as many ORFs. Both of these functions are called repeatedly within the ORFfinder() function for the different start codons ATG, TTG, GTG. Within this function, the ORFcheck() function also runs to determine how many genes the ORF finder finds and how many extraneous ORFs it finds

In [1]:
import numpy as np
import pandas as pd

<font color='blue'>Cell 2 
Reading data

In [2]:
#Importing gene position data 
#The first element of each row is the start position of the gene
#The second is the end position of the gene
all_gene_data = pd.read_csv('all_gene_positions.txt',header=None).to_numpy()    
gene_position = all_gene_data[0:2269,:]     #Data for forward direction
gene_position_r = all_gene_data[2269:,:]    #Data for reverse complement

#Reading full forward direction genome
with open('full_sequence.fasta', 'r') as file:
    full_seq = file.read().replace('\n', '')

#Remove first line (description of the data)
full_seq = full_seq[71:]

#Reading full reverse complement direction genome
with open('rev_comp.fasta', 'r') as file:
    full_rev = file.read().replace('\n', '')

#Remove the first line
full_rev = full_rev[82:]

<font color='blue'>Cell 3
Data set splitting

In [3]:
#The "sample" function will sort the data rows (proteins) randomly
def split_dataset_test_train(data,train_size):
  #Shuffles the data rows randomly
  data = data.sample(frac=1).reset_index(drop=True)
  #Training data is all rows up to the length of the data set multiplied by 'train_size' (between 0 and 1)
  training_data = data.iloc[:int(train_size * len(data))].reset_index(drop=True)
  #Test data is all rows after length of the data set multiplied by 'train_size'
  testing_data = data.iloc[int(train_size * len(data)):].reset_index(drop=True)
  return training_data, testing_data




# ORF finder

<font color='blue'>Cell 4
Finding codon positions

codons: This function will find the positions at which a start codon or stop codon appears. This function will return two lists (start and stop), each with three sublists that contain the found positions in three frames. 

Inputs: 
-   seq: The sequence being considered, ie. forward or reverse complement sequences (full_seq and full_rev respectively)
-   start_c: The start codon considered. This function is called from another function that iterates through ATG, GTG, TTG

Inside the function:
-   stops: The stop codons that will be found
-   lst1: A list that will contain the start codon positions
-   lst2: A list that will contain the stop codon positions
      - Both lst1 and lst2 are a list of 3 lists where each sublist is the list of positions for each frame.
-   while(start < 3): For values of iterator 'start' of 0, 1, and 2, look at all codons (in frame) from start to the end of the sequence. If the codon is a match for the start codon of interest, add it to the corresponding lst1 list. If the codon is a match for any of the stop codons, add it to the corresponding list in lst2. Once the end of the sequence is reached, increment the 'start' iterator to look for codons in the next frame.

Outputs:
-    Returns the lists lst1 and lst2

In [4]:
def codons(seq,start_c):
  stops = ["TAA","TGA","TAG"] 
  lst1 = [] #List for the start codons
  lst2 = [] #List for the stop codons
  start = 0 #The start position of the sequence.
  #initializes the lists for 3 to have 3 sublists with positions for each frame
  for i in range (3):
      lst1.append([])
      lst2.append([])
  #Add to the lists the positions of the start and stop codons
  
  while (start < 3): #(seq and start < 3)

      for i in range(start,len(seq),3):
          codon = seq[i:i+3] #The codon is 3 nucleotides.
          #print codon+ "\t"
          if (codon == start_c): #If the codon is  a start codon.
              lst1[start].append(i+1) #The position of the start codon.

          if (codon in stops): #if the codon is a stop codon.
              lst2[start].append(i+1) #The position of the stop codon.


      start += 1 #promotes the starting position.
  return lst1,lst2

ORF: Pairs the start and stop codon positions found with the codons function to form valid ORF start stop pairs. 

Inputs: 
-   lst1 and lst2 that are the outputs of the 'codons()' function
-   min_len: The minimum length used for determining valid ORFs

Inside the function:
-   ORFstarts and ORFstops: Lists that will contain the starts and stops of the determined valid ORFs
-   all_frames: When looking for ORFs, all start codon positions for all frames will be put into one list. This allows to scan for valid ORF starts sequentially across all frames.
-   start_min: Starting from zero, this variable is updated to dictate the position from which the next ORF start will be considered. After a start codon has been paired with its stop codons, this variable is updated with the min_len and the look_back. 
-   look_back: This parameter is subtracted from the start_min variable to account for the possibility of overlapping ORFs

Outputs: 
-   ORFstarts and ORFstops: 
-   found_frame: A list containg the frames for all the valid start codons (only used for diagnostic purposes)

<font color='blue'>Cell 5
Determining valid ORFs

In [5]:

def ORF(lst1,lst2,min_len):
  ORFstarts = []    #Contains the start positions of valid ORFs
  ORFstops = []     #Contains the end positions of valid ORFs
  found_frame = []  #Was only used for diagnostic purposes
  curr_frame = 0    
  start_min = 0     #The position from which the next ORF start will be considered
  look_back = 5     #Subtracted from start_min to account for overlapping ORFs
  max_fr_len = max([len(i) for i in lst1])

  #Putting all start codons for all frames into one list
  all_frames = lst1[0] + lst1[1] + lst1[2]
  #Sorting the list of all start codon positions
  all_frames.sort()
  

  for s_cod in all_frames:
    #To be considered valid, the start codon position must be greater than start_min
    if s_cod > start_min:
      for s_frame in range(len(lst1)):  #Find frame of start codon
        if s_cod in lst1[s_frame]:
          #frame = lst1[s_frame].index(s_cod)
          frame = s_frame   # 'frame' is the frame of the current start codon 
          found_frame.append(frame)
          break
      stop_count = 0    #Each valid start codon is paired with 3 stops 

      #Once the frame of the next start codon is determined, look for valid stop codons in the same frame
      for stp_cod in lst2[frame]:
        #In the list of stop codons, only consider stops that will satisfy the min_len condition
        if stp_cod > s_cod + min_len:
          #If the stop satisfies this condition, enter the start stop pair into the corresponding lists
          ORFstarts.append(s_cod)
          ORFstops.append(stp_cod)
          #Increment the stop_count to get 2 more stop codons for the start codon
          stop_count +=1 
        if stop_count > 2:
          #Once 3 stop codons found for the start, update start_min 
          start_min = start_min + min_len - look_back
          #Break out of the stop codons loop and look for the next valid start codon
          break



  return ORFstarts, ORFstops, found_frame

<font color='blue'>Cell 6
ORF checking function

ORFcheck: Checks which of the ORF start stop pairs are coding (ie. found in the gene position data) and which are non coding. 

Inputs: 
-   ORFstarts and ORFstops lists from the ORF() function
-   gene_pos: The gene position data (either gene_position or gene_position_r)
-   rev: A true/false parameter that is set to true if looking at the reverse complement (with gene_position_r)

Inside the function:
-   corr: The number of found ORFs that are contained in the gene position data 
-   corORF: An array of the ORFs (start and end position) that are contained in the gene position data
-   incorORF: An array of the ORFs (start and end position) that are not contained in the gene position data (non coding ORFs)

In [6]:
#ORF finder part 3: Determining which found ORFs are correct and incorrect

def ORFcheck(ORFstr,ORFstp,gene_pos,rev):
  corr = 0        #The number of found ORFs that are contained in the gene position data 
  corORF = np.empty((0,2))   #An array of the ORFs (start and end position) that are contained in the gene position data
  incorORF = np.empty((0,2)) #An array of the ORFs (start and end position) that are not in the gene position data (non coding ORFs)
  wrong_flag = 1    #A flag variable that is initially set to 1, indicating a non coding ORF

  #Iterate through all codons in the ORFstart list
  for i in range(len(ORFstr)):
    wrong_flag = 1
    #OC is current ORF start stop pair that is being tested
    #If rev, subtract one from the start position (different indexing)
    if rev:
      OC = np.array([ORFstr[i]-1, ORFstp[i]+2])
    else:
      OC = np.array([ORFstr[i], ORFstp[i]+2])
    
    #The following 2 conditions check to see if OC is contained in the gene position data
    if len(np.where(gene_pos==OC)[0]) == 2:
      if np.where(gene_pos==OC)[0][0] == np.where(gene_pos==OC)[0][1]:
        #If the two conditions are met, then increment 'corr' and enter the ORF start and stop into the corORF array
        corr += 1
        OC = OC.reshape(1,-1)
        corORF = np.concatenate((corORF,OC), axis=0)
        #Set the wrong flag to 0 to avoid passing the next condition
        wrong_flag = 0
    
    #If the two conditions are not true, ie. wrong_flag is still 1, enter the ORF pair into the incorORF array
    if wrong_flag == 1:
      OC = OC.reshape(1,-1)
      incorORF = np.concatenate((incorORF,OC), axis=0)
  return corr, corORF, incorORF

<font color='blue'>Cell 7
Running the previous functions with start codons from a list

ORFfinder: This function simply runs the functions defined above for different start codons ATG, GTG, TTG and returns the arrays of all coding and non coding ORFs found

Inputs: 
-   dta: Either full_seq or full_rev (genome sequence)
-   nucleotides: A parameter to limit the ORFfinder search (ie. nucleotides = 10000 will only look at the first 10000 nucleotides)
-   sel_gen: The gene position data that is also limited by the 'nucleotides' parameter
-   rev and min_len as defined in previous functions



In [7]:
def ORFfinder(dta,nucleotides, sel_gen, rev,min_len):
  all_corORFs = np.empty((0,2))
  all_incorORFs = np.empty((0,2))
  start_codons = ["ATG","GTG","TTG"]
  for start_cod in range(len(start_codons)):
    lst1, lst2 = codons(dta[0:nucleotides], start_codons[start_cod])
    ORFstarts, ORFstops, found_frame = ORF(lst1,lst2,min_len)
    corr, corORF, incorORF = ORFcheck(ORFstarts,ORFstops, sel_gen, rev)
    all_corORFs = np.concatenate((all_corORFs,corORF),axis=0)
    all_incorORFs = np.concatenate((all_incorORFs,incorORF),axis=0)

  return all_corORFs, all_incorORFs

# ORF Finder



<font color='blue'>Cell 8
Finding ORFs on a section of the E. Coli genome

genome_section: This function takes inputs from the user and runs the ORFfinder function. This function outputs in text how many coding ORFs and non coding ORFs were found


In [8]:
def genome_section():
  #Get the necessary parameters from the user
  nucleotides = input("Please enter the number of nucleotides to analyze: ")
  min_len = input("Please enter the minimum length of ORFs to consider: ")
  min_len = int(min_len)
  nucleotides = int(nucleotides)
  #Limit the forward and reverse complement using the 'nucleotides' parameter
  forward_section = full_seq[0:nucleotides]
  reverse_section = full_rev[0:nucleotides]

  #Limiting the gene position data by the 'nucleotides' parameter 
  selected_genes_f = np.array([gene_position[i] for i in range(len(gene_position)) if gene_position[i,1] < nucleotides])
  selected_genes_r = np.array([gene_position_r[i] for i in range(len(gene_position_r)) if gene_position_r[i,1] < nucleotides])
  print("-----------------------------------------------------------\n")
  print("There are", str(len(selected_genes_f)), "genes on the forward strand")
  print("There are", str(len(selected_genes_r)), "genes on the reverse strand")
  print("-----------------------------------------------------------\n")

  #Running the ORFfinder function on the forward and reverse complement with the entered parameters
  f_corr, f_incorr = ORFfinder(full_seq, nucleotides,selected_genes_f, rev=False,min_len=min_len)
  r_corr, r_incorr = ORFfinder(full_rev, nucleotides,selected_genes_r, rev=True,min_len=min_len)

  #Printing the number of coding ORFs and non coding ORFs found
  print("Forward:")
  print("\t",str(len(f_corr)), "found correctly")
  print("\t",str(len(f_incorr)), "additional ORFs found")

  print("Reverse:")
  print("\t",str(len(r_corr)), "found correctly")
  print("\t",str(len(r_incorr)), "additional ORFs found")

  all_f_pred = np.concatenate((f_corr,f_incorr)).astype(int)
  all_r_pred = np.concatenate((r_corr,r_incorr)).astype(int)


  return all_f_pred, all_r_pred


all_f_pred, all_r_pred = genome_section()

Please enter the number of nucleotides to analyze: 10000
Please enter the minimum length of ORFs to consider: 100
-----------------------------------------------------------

There are 6 genes on the forward strand
There are 3 genes on the reverse strand
-----------------------------------------------------------

Forward:
	 5 found correctly
	 883 additional ORFs found
Reverse:
	 1 found correctly
	 898 additional ORFs found


In [ ]:
#stop

<font color='blue'>Cell 9
Running the previous functions with start codons from a list

In [ ]:
genome_filename = 'genome1.fasta'
def test_genome_ORFs(lim):
  test_genome = pd.read_csv(genome_filename,header=None).to_numpy()[0][0]
  test_genome = test_genome[0:int(len(test_genome)*lim)]
  #Finding reverse complement  
  test_genome_r = test_genome[::-1]
  tst_gen_rcomp = ''
  for i in test_genome_r:
    if i == 'A':
      tst_gen_rcomp += 'T'
    elif i == 'T':
      tst_gen_rcomp += 'A'
    elif i == 'C':
      tst_gen_rcomp += 'G'
    else:
      tst_gen_rcomp += 'C'
  test_genome_r = tst_gen_rcomp

  min_len = input("Please enter the minimum length of ORFs to consider: ")
  print('\n')
  min_len = int(min_len)

  start_codons = ["ATG","GTG","TTG"]
  for start_cod in range(len(start_codons)):
    lst1, lst2 = codons(test_genome, start_codons[start_cod])
    ORFstarts, ORFstops, found_frame = ORF(lst1,lst2,min_len)
    forward_ORFs = [[ORFstarts[i],ORFstops[i]] for i in range(len(ORFstarts))]


    lst1_r, lst2_r = codons(test_genome_r, start_codons[start_cod])
    ORFstarts_r, ORFstops_r, found_frame = ORF(lst1_r,lst2_r,min_len)

    reverse_ORFs = [[ORFstarts_r[i],ORFstops_r[i]] for i in range(len(ORFstarts_r))]

    forward_ORFs = pd.DataFrame(forward_ORFs)
    forward_ORFs = forward_ORFs.rename(columns={0: "ORF start location", 1: "ORF stop location"})
    reverse_ORFs = pd.DataFrame(reverse_ORFs)
    reverse_ORFs = reverse_ORFs.rename(columns={0: "ORF start location", 1: "ORF stop location"})
  print("-----------------------------------------------------------\n")
  print("ORFs found on the forward direction:            ", len(ORFstarts))
  print("ORFs found on the reverse complement direction: ", len(ORFstarts_r))

  return forward_ORFs, reverse_ORFs

size_limit = float(input("Enter a value between 0 and 1 to limit the genome"))
forward_ORFs, reverse_ORFs = test_genome_ORFs(size_limit)

print("-----------------------------------------------------------")

print("\nThe first 5 ORFs found on the forward direction: \n")
print(forward_ORFs.iloc[0:5,:])


print("-----------------------------------------------------------")

print("\nThe first 5 ORFs found on the reverse complement direction: \n")
print(reverse_ORFs.iloc[0:5,:])


Enter a value between 0 and 1 to limit the genome.2
Please enter the minimum length of ORFs to consider: 100


-----------------------------------------------------------

ORFs found on the forward direction:             8422
ORFs found on the reverse complement direction:  8402
-----------------------------------------------------------

The first 5 ORFs found on the forward direction: 

   ORF start location  ORF stop location
0                  10                337
1                  10                346
2                  10                364
3                 111               1299
4                 111               2466
-----------------------------------------------------------

The first 5 ORFs found on the reverse complement direction: 

   ORF start location  ORF stop location
0                  40                196
1                  40                226
2                  40                370
3                 167               1412
4                 167             